
This file is to show a sample of the code used for creating the scenario 4 dataset. The following code uses the scapy package to import and create pcap files. A reference to the scapy guide that was used is located at:

https://scapy.readthedocs.io/en/latest/


In [ ]:
pip install scapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 14.2 MB/s 
  Created wheel for scapy: filename=scapy-2.4.5-py2.py3-none-any.whl size=1261555 sha256=847a457e00b1e270256bc29cf451cce5f5683a6f24c087da7a4c4c9bbb7098aa
  Stored in directory: /root/.cache/pip/wheels/b9/6e/c0/0157e466a5e02d3ff28fc7587dff329b4a967a23b3f9b11385
Successfully built scapy


In [ ]:
import pandas as pd
import numpy as np
from scapy.all import *
from scapy.layers.http import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
attack_list = ['Week1_Wednesday_Neptune', 
               'Week1_Wednesday_Smurf', 
               'Week1_Thursday_Pod', 
               'Week1_Friday_Teardrop', 
               'Week2_Wednesday_Land', 
               'Week2_Friday_Back', 
               'Week2_Friday_Syslog', 
               'Week3_Wednesday_Smurf', 
               'Week3_Friday_Land', 
               'Week4_Tuesday_Pod', 
               'Week4_Tuesday_Teardrop', 
               'Week4_Thursday_Syslog', 
               'Week5_Wednesday_Syslog', 
               'Week5_Thursday_Land', 
               'Week6_Monday_Neptune', 
               'Week6_Wednesday_Back', 
               'Week6_Thursday_Pod1']

In [ ]:
labels = []
for attack in attack_list:
  temp = pd.read_csv('/content/drive/MyDrive/ECE 697 Project/1. datasets/DARPA 1998/DARPA98_Attacks/Labels_Truncated/DARPA98_Training_'+attack+'_Attack_Labels.csv', header=None)
  temp = np.array(temp).reshape(1, -1)
  labels.append(temp)

In [ ]:
total_pkts = 0
total_attack_pkts = 0
for i in range(len(labels)):
  total_pkts += np.shape(labels[i])[1]
  total_attack_pkts += np.sum(labels[i])

In [ ]:
# number of benign packets in DARPA98 attack pcaps that need to be replaced with SUEE8 benign packet
total_benign_pkts_to_replace = total_pkts - total_attack_pkts 

In [ ]:
# number of SUEE8 benign packets to add to make final pcap contain 100000 packets
total_benign_pkts_to_add = 100000 - total_benign_pkts_to_replace - total_attack_pkts

In [ ]:
# total number of SUEE8 benign packets needed for final pcap
total_benign_pkts_needed = total_benign_pkts_to_add + total_benign_pkts_to_replace 

In [ ]:
# randomize the number of benign packets that are inserted between attacks
rand = np.random.random(size = len(labels) + 1)
x = total_benign_pkts_to_add / sum(rand)
rand = np.round(x*rand)
rand = np.array(rand, dtype = int)

In [ ]:
# create new pcap and label files
i = 0
DARPA98_Attacks_SUEE8_Benign_Labels = []
SUEE8_benign_pcap = PcapReader('/content/drive/MyDrive/ECE 697 Project/1. datasets/SUEE 17/SUEE Benign Traffic/SUEE8_Benign.pcap')
for attack in attack_list:
  for j in range(rand[i]):
    benign_pkt = SUEE8_benign_pcap.next()
    wrpcap('DARPA98_Attacks_SUEE8_Benign.pcap', benign_pkt, append=True)
    DARPA98_Attacks_SUEE8_Benign_Labels.append(0)
  DARPA98_attack_pcap = PcapReader('/content/drive/MyDrive/ECE 697 Project/1. datasets/DARPA 1998/DARPA98_Attacks/Attacks_Truncated/DARPA98_Training_'+attack+'_Attack.pcap')
  for k in range(np.shape(labels[i])[1]):
    if labels[i][0,k] == 1:
      attack_pkt = DARPA98_attack_pcap.next()
      wrpcap('DARPA98_Attacks_SUEE8_Benign.pcap', attack_pkt, append=True)
      DARPA98_Attacks_SUEE8_Benign_Labels.append(1)
    else:
      attack_pkt = DARPA98_attack_pcap.next()
      benign_pkt = SUEE8_benign_pcap.next()
      wrpcap('DARPA98_Attacks_SUEE8_Benign.pcap', benign_pkt, append=True)
      DARPA98_Attacks_SUEE8_Benign_Labels.append(0)
  i += 1
for j in range(rand[i]):
  benign_pkt = SUEE8_benign_pcap.next()
  wrpcap('DARPA98_Attacks_SUEE8_Benign.pcap', benign_pkt, append=True)
  DARPA98_Attacks_SUEE8_Benign_Labels.append(0)
pd.DataFrame(DARPA98_Attacks_SUEE8_Benign_Labels).to_csv('DARPA98_Attacks_SUEE8_Benign_Labels.csv', header=None, index=False)
